# The notebook applying LSTM-based approach proposed by Rieger et al. (2023)

Rieger, Laura Hannemose, et al. “Uncertainty-Aware and Explainable Machine Learning for Early Prediction of Battery Degradation Trajectory.” Digital Discovery, vol. 2, no. 1, 2023, pp. 112–22. DOI: https://doi.org/10.1039/D2DD00067A.

In [1]:
import pickle as pkl
import sys
import warnings


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.utils.data
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import KFold,GroupKFold

from torch import optim
from torch import nn
from torch.utils.data import DataLoader, TensorDataset,Subset
import torch.nn.functional as F

Import the preprocessed training data for the LSTM network

In [2]:
train_X_lstm = np.load('Data/train_X_lstm.npy')
train_Y_lstm = np.load('Data/train_Y_lstm.npy')
train_C_lstm = np.load('Data/train_C_lstm.npy')
with open('Data/train_cell_lstm.pkl', 'rb') as f: 
    train_cell_lstm = pkl.load(f)

# Scale the output data to be between -1 and 1, with a minimum value of 0.95 and a maximum value of 1.0
min_val = 0.95
max_val = 1.0
capacity_output_scaler = MinMaxScaler(
    (-1, 1),
    clip=False).fit(np.maximum(np.minimum(train_Y_lstm[:, 0:1], max_val), min_val))
train_Y_lstm[:, 0:1] = capacity_output_scaler.transform(train_Y_lstm[:, 0:1])
min_max_range = max_val - min_val
# Build the dataset
dataset = TensorDataset(torch.tensor(train_X_lstm, dtype=torch.float32),
                        torch.tensor(train_Y_lstm, dtype=torch.float32),
                        torch.tensor(train_C_lstm, dtype=torch.float32))

In [ ]:
plt.hist(train_Y_lstm[:, 0:1], bins=np.arange(-1, 1, 0.01))
plt.show()

# Training the LSTM model

Build the LSTM model class

In [18]:
class Uncertain_LSTM_new(nn.Module):
    def __init__(
        self,
        num_in=1,
        num_augment=3,
        num_hidden=100,
        seq_len=5,
        n_layers=2,
        dropout=0.0,
        num_hidden_lstm=-1,
    ):
        super(Uncertain_LSTM_new, self).__init__()
        self.hidden_dim = num_hidden
        if num_hidden_lstm == -1:
            num_hidden_lstm = num_hidden
        self.hidden_dim_lstm = num_hidden_lstm
        self.seq_len = seq_len
        self.n_layers = n_layers

        # LSTM layer
        self.lstm1 = nn.LSTM(
            num_in, self.hidden_dim_lstm, num_layers=n_layers, batch_first=True
        )

        # Fully connected layers
        self.linear1 = nn.Linear(self.hidden_dim_lstm + num_augment, num_hidden)
        self.mean_state_layer = nn.Linear(num_hidden, 1)
        self.var_state_layer = nn.Linear(num_hidden, 1)

        # Dropout layer
        self.drop_layer = nn.Dropout(p=dropout)

        # Initialize weights
        self.init_weights()

    def init_weights(self):
        """Initializes the weights of the network."""
        # Initialize LSTM weights
        for name, param in self.lstm1.named_parameters():
            if 'weight_ih' in name:
                nn.init.xavier_uniform_(param)  # Xavier initialization for input weights
            elif 'weight_hh' in name:
                nn.init.orthogonal_(param)  # Orthogonal initialization for hidden state weights
            elif 'bias' in name:
                param.data.fill_(0)  # Zero initialize biases

        # Initialize fully connected layers
        nn.init.xavier_uniform_(self.linear1.weight)
        self.linear1.bias.data.fill_(0)

        nn.init.xavier_uniform_(self.mean_state_layer.weight)
        self.mean_state_layer.bias.data.fill_(0)

        nn.init.xavier_uniform_(self.var_state_layer.weight)
        self.var_state_layer.bias.data.fill_(-2.0)  # Helps keep variance low initially

    def forward(self, x, c):
        x, _ = self.lstm1(x)

        x = x[:, -1].view(-1, self.hidden_dim_lstm)

        x = torch.cat((x, c), dim=1)
        x = F.relu(self.linear1(x))
        x = self.drop_layer(x)

        mean_state = torch.tanh(self.mean_state_layer(x))
        var_state = F.softplus(self.var_state_layer(x)) + 1e-6

        return mean_state, var_state

Define the NLL loss function

In [19]:
def nll_loss(target, mean, var):
    # print(target.shape, mean.shape, var.shape)
    nll = 0.5 * (torch.log(var) + torch.square(target - mean) / ((var)))
    return torch.sum(nll)

def mse_loss(target, mean):
    return 0.5*torch.sum(torch.square(target - mean))

# Define the NLL loss function with warmup (i.e., ignore the variance)
def nll_loss_warmup(target, mean, var, warmup=False):
    if warmup:
        # Ignore the variance resulting in a simple MSE loss
        var_log = 0
        var = 1
        nll = 0.5 * (var_log + torch.square(target - mean) / var)
    else:
        nll = 0.5 * (torch.log(var) + torch.square(target - mean) / var)
    return torch.sum(nll)

Hyperparameters and configurations

In [20]:
# Hyperparameters for LSTM model
batch_size = 128
max_epochs = 1000
lr = 1e-3
hidden_size = 16
hidden_size_lstm = 32
num_layers = 2
cur_patience = 0
max_patience = 5
patience_delta = 0
dropout = 0.0

# Additional configurations
sequence_length = 10
num_augment = train_C_lstm.shape[1]
input_size = train_X_lstm.shape[2]
num_folds = 10
num_ensemble = 5
warmup_epochs = 10 # Number of epochs to ignore the variance
stabilize_epochs = 10 # Number of epochs to stabilize the variance

Train ensemble of 5 models, 10-fold CV

In [ ]:
# kfold = KFold(n_splits=num_folds, shuffle=True, random_state=42)
kf = GroupKFold(n_splits=num_folds)
kfold = kf.split(train_X_lstm, groups=train_cell_lstm)
model_state_dict = {}
for fold,(train_idx,val_idx) in enumerate(kfold):
    model_state_dict[fold] = []
    print(f"Fold {fold + 1}/{num_folds}")
    # Subset the TensorDataset for training and validation
    train_subset = Subset(dataset, train_idx)
    val_subset = Subset(dataset, val_idx)

    # Create the DataLoader for training and validation
    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)

    for i in range(num_ensemble):
        print(f"Fold {fold+1}: Ensemble {i+1}/{num_ensemble}")
        # Initialize the model
        model = Uncertain_LSTM_new(
            num_in=input_size,
            num_augment=num_augment,
            num_hidden=hidden_size,
            seq_len=sequence_length,
            n_layers=num_layers,
            dropout=dropout,
            num_hidden_lstm=hidden_size_lstm,
        )
        # Initialize the warmup flag
        warmup = True
        for param in model.var_state_layer.parameters():
            param.requires_grad = False

        # Initialize the optimizer
        optimizer = optim.Adam(model.parameters(), lr=lr)

        # Initialize the loss function
        criterion = nll_loss_warmup

        # Initialize the early stopping variables
        best_loss = np.inf
        cur_patience = 0

        for epoch in range(max_epochs):
            model.train()
            tr_loss = 0
            tr_loss_mse = 0
            # Reset warmup flag after warmup_epochs
            if epoch == warmup_epochs:
                warmup = False
                for param in model.var_state_layer.parameters():
                    param.requires_grad = True

            for i, (x, y, c) in enumerate(train_loader):
                optimizer.zero_grad()
                mean, var = model(x, c)
                loss = criterion(y[:,0], mean[:,0], var[:,0], warmup)
                loss.backward()
                optimizer.step()
                tr_loss += loss.item()
                tr_loss_mse += mse_loss(y[:,0], mean[:,0]).item()

            tr_loss /= len(train_loader.dataset)
            tr_loss_mse /= len(train_loader.dataset)

            model.eval()
            with torch.no_grad():
                val_loss = 0
                val_loss_mse = 0
                for i, (x_val, y_val, c_val) in enumerate(val_loader):
                    mean_val, var_val = model(x_val, c_val)
                    val_loss += criterion(y_val[:,0], mean_val[:,0], var_val[:,0], warmup).item()
                    val_loss_mse += mse_loss(y_val[:,0], mean_val[:,0]).item()
                val_loss /= len(val_loader.dataset)
                val_loss_mse /= len(val_loader.dataset)

                print(f"Epoch {epoch+1}/{max_epochs}, Train Loss: {tr_loss:.4f}, Val Loss: {val_loss:.4f}, Train MSE: {tr_loss_mse:.4f}, Val MSE: {val_loss_mse:.4f}")

                if (
                        val_loss + patience_delta < best_loss
                        and epoch > warmup_epochs + stabilize_epochs
                    ): 
                    best_loss = val_loss
                    best_model = model.state_dict()
                    cur_patience = 0
                else:
                    if epoch > warmup_epochs + stabilize_epochs:
                        cur_patience += 1
                        if cur_patience >= max_patience:
                            break

        model_state_dict[fold].append(best_model)

In [22]:
with open('model_state_dict.pth', 'wb') as f:
    torch.save(model_state_dict, f)

# Evaluate the prediction results

In [ ]:
# Load the model state dictionary
with open('model_state_dict.pth', 'rb') as f:
    model_state_dict = torch.load(f)

training_cells = pd.read_csv("../Data_preprocessing/training.csv",header=None).to_numpy(dtype=str).reshape(-1,).tolist()
test_in_cells = pd.read_csv("../Data_preprocessing/test_in.csv",header=None).to_numpy(dtype=str).reshape(-1,).tolist()
test_out_cells = pd.read_csv("../Data_preprocessing/test_out.csv",header=None).to_numpy(dtype=str).reshape(-1,).tolist()

# Load the preprocessed data for evaluation
train_X_lstm_eval = np.load('Data/train_X_lstm_eval.npy')
with open('Data/train_remaining_eval.pkl', 'rb') as f:
    train_remaining_eval = pkl.load(f)
with open('Data/train_C_lstm_eval.pkl', 'rb') as f:
    train_C_lstm_eval = pkl.load(f)

test_in_X_lstm_eval = np.load('Data/test_in_X_lstm_eval.npy')
with open('Data/test_in_remaining_eval.pkl', 'rb') as f:
    test_in_remaining_eval = pkl.load(f)
with open('Data/test_in_C_lstm_eval.pkl', 'rb') as f:
    test_in_C_lstm_eval = pkl.load(f)


test_out_X_lstm_eval = np.load('Data/test_out_X_lstm_eval.npy')
with open('Data/test_out_remaining_eval.pkl', 'rb') as f:
    test_out_remaining_eval = pkl.load(f)
with open('Data/test_out_C_lstm_eval.pkl', 'rb') as f:
    test_out_C_lstm_eval = pkl.load(f)

In [68]:
Q_train_true = {}
Q_test_in_true = {}
Q_test_out_true = {}

Q_train_pred_5_all = {}
Q_test_in_pred_5_all = {}
Q_test_out_pred_5_all = {}

std_train_pred_5_all = {}
std_test_in_pred_5_all = {}
std_test_out_pred_5_all = {}

Q_train_pred_5_ensemble = {}
Q_test_in_pred_5_ensemble = {}
Q_test_out_pred_5_ensemble = {}

std_train_pred_5_ensemble = {}
std_test_in_pred_5_ensemble = {}
std_test_out_pred_5_ensemble = {}


all_pred_samples = {}

for fold in range(10):
    Q_train_pred_5_all[fold] = {}
    std_train_pred_5_all[fold] = {}
    all_pred_samples[fold] = []
    for i in range(5):
        print(f"Fold {fold+1}: Ensemble {i+1}/{num_ensemble}")
        # Initialize the model
        model = Uncertain_LSTM_new(
            num_in=input_size,
            num_augment=num_augment,
            num_hidden=hidden_size,
            seq_len=sequence_length,
            n_layers=num_layers,
            dropout=dropout,
            num_hidden_lstm=hidden_size_lstm,
        )
        # Load the trained model parameters
        model.load_state_dict(model_state_dict[fold][i])
        
        model.eval()
        with torch.no_grad():
            # Evaluate the model on the training set
            for j in range(train_X_lstm_eval.shape[0]):
                # Indexing the cell
                cell = training_cells[j]
                if cell not in Q_train_pred_5_all[fold]:
                    Q_train_pred_5_all[fold][cell] = []
                    std_train_pred_5_all[fold][cell] = []
                
                # Get the observed capacity
                x = torch.tensor(train_X_lstm_eval[j:j+1], dtype=torch.float32)
                x_pred = x.clone()
                # Get predicted remaining capacity
                Q_pred = []
                std_pred = []
                for k in range(len(train_remaining_eval[cell])):
                    # Find the features with the corresponding Ah-throughput location
                    c = torch.tensor(train_C_lstm_eval[cell][k].reshape(1,-1), dtype=torch.float32)
                    # Predict the remaining capacity ratio and variance
                    seq = x_pred[:,-10:,:]
                    mean_scaled, var_scaled = model(seq, c) # outputs are (ratio between existing and new capacity, log variance)
                    # Squeeze the tensor to remove the batch dimension
                    mean_scaled = mean_scaled.squeeze()
                    all_pred_samples[fold].append(mean_scaled.item())
                    var_scaled = var_scaled.squeeze()
                    # Inverse transform the scaled ratio to the original scale
                    var_ratio = var_scaled*(min_max_range**2)
                    ratio = torch.tensor(capacity_output_scaler.inverse_transform(np.array(mean_scaled.item()).reshape(-1,1)), dtype=torch.float32)

                    # Append the predicted remaining capacity ratio
                    x_pred = torch.cat((x_pred, ratio[:]*(x_pred[:,-1,-1])[None,:,None]), dim=1)

                    # Append the predicted standard deviation
                    std_pred.append(np.sqrt(var_ratio.item())*(x_pred[:,-1,-1])[None,:,None].item())  

                # Concatenate the true remaining capacity with the observed capacity
                x_true = torch.concat((x, torch.tensor(train_remaining_eval[cell], dtype=torch.float32)[None,:,None]), dim=1)
                if cell not in Q_train_true:
                    Q_train_true[cell] = x_true.squeeze().numpy()
                Q_train_pred_5_all[fold][cell].append(x_pred.squeeze().numpy())
                std_train_pred_5_all[fold][cell].append(np.array(std_pred))

with open('Q_train_pred_5_all.pkl', 'wb') as f:
    pkl.dump(Q_train_pred_5_all, f)

with open('std_train_pred_5_all.pkl', 'wb') as f:
    pkl.dump(std_train_pred_5_all, f)

with open('Q_train_true.pkl', 'wb') as f:
    pkl.dump(Q_train_true, f)




Fold 1: Ensemble 1/5
Fold 1: Ensemble 2/5
Fold 1: Ensemble 3/5
Fold 1: Ensemble 4/5
Fold 1: Ensemble 5/5
Fold 2: Ensemble 1/5
Fold 2: Ensemble 2/5
Fold 2: Ensemble 3/5
Fold 2: Ensemble 4/5
Fold 2: Ensemble 5/5
Fold 3: Ensemble 1/5
Fold 3: Ensemble 2/5
Fold 3: Ensemble 3/5
Fold 3: Ensemble 4/5
Fold 3: Ensemble 5/5
Fold 4: Ensemble 1/5
Fold 4: Ensemble 2/5
Fold 4: Ensemble 3/5
Fold 4: Ensemble 4/5
Fold 4: Ensemble 5/5
Fold 5: Ensemble 1/5
Fold 5: Ensemble 2/5
Fold 5: Ensemble 3/5
Fold 5: Ensemble 4/5
Fold 5: Ensemble 5/5
Fold 6: Ensemble 1/5
Fold 6: Ensemble 2/5
Fold 6: Ensemble 3/5
Fold 6: Ensemble 4/5
Fold 6: Ensemble 5/5
Fold 7: Ensemble 1/5
Fold 7: Ensemble 2/5
Fold 7: Ensemble 3/5
Fold 7: Ensemble 4/5
Fold 7: Ensemble 5/5
Fold 8: Ensemble 1/5
Fold 8: Ensemble 2/5
Fold 8: Ensemble 3/5
Fold 8: Ensemble 4/5
Fold 8: Ensemble 5/5
Fold 9: Ensemble 1/5
Fold 9: Ensemble 2/5
Fold 9: Ensemble 3/5
Fold 9: Ensemble 4/5
Fold 9: Ensemble 5/5
Fold 10: Ensemble 1/5
Fold 10: Ensemble 2/5
Fold 10: En

In [ ]:
for fold in range(10):
    Q_test_in_pred_5_all[fold] = {}
    std_test_in_pred_5_all[fold] = {}
    all_pred_samples[fold] = []
    for i in range(5):
        print(f"Fold {fold+1}: Ensemble {i+1}/{num_ensemble}")
        # Initialize the model
        model = Uncertain_LSTM_new(
            num_in=input_size,
            num_augment=num_augment,
            num_hidden=hidden_size,
            seq_len=sequence_length,
            n_layers=num_layers,
            dropout=dropout,
            num_hidden_lstm=hidden_size_lstm,
        )
        # Load the trained model parameters
        model.load_state_dict(model_state_dict[fold][i])
        
        model.eval()
        with torch.no_grad():
            # Evaluate the model on the training set
            for j in range(test_in_X_lstm_eval.shape[0]):
                # Indexing the cell
                cell = test_in_cells[j]
                if cell not in Q_test_in_pred_5_all[fold]:
                    Q_test_in_pred_5_all[fold][cell] = []
                    std_test_in_pred_5_all[fold][cell] = []
                
                # Get the observed capacity
                x = torch.tensor(test_in_X_lstm_eval[j:j+1], dtype=torch.float32)
                x_pred = x.clone()
                # Get predicted remaining capacity
                Q_pred = []
                std_pred = []
                for k in range(len(test_in_remaining_eval[cell])):
                    # Find the features with the corresponding Ah-throughput location
                    c = torch.tensor(test_in_C_lstm_eval[cell][k].reshape(1,-1), dtype=torch.float32)
                    # Predict the remaining capacity ratio and variance
                    seq = x_pred[:,-10:,:]
                    mean_scaled, var_scaled = model(seq, c) # outputs are (ratio between existing and new capacity, log variance)
                    # Squeeze the tensor to remove the batch dimension
                    mean_scaled = mean_scaled.squeeze()
                    all_pred_samples[fold].append(mean_scaled.item())
                    var_scaled = var_scaled.squeeze()
                    # Inverse transform the scaled ratio to the original scale
                    ratio = torch.tensor(capacity_output_scaler.inverse_transform(np.array(mean_scaled.item()).reshape(-1,1)), dtype=torch.float32)
                    var_ratio = var_scaled*(min_max_range**2)
                    # Append the predicted remaining capacity ratio
                    x_pred = torch.cat((x_pred, ratio[:]*(x_pred[:,-1,-1])[None,:,None]), dim=1)
                    
                    # Append the predicted standard deviation
                    std_pred.append(np.sqrt(var_ratio.item())*(x_pred[:,-1,-1])[None,:,None].item())  

                # Concatenate the true remaining capacity with the observed capacity
                x_true = torch.concat((x, torch.tensor(test_in_remaining_eval[cell], dtype=torch.float32)[None,:,None]), dim=1)
                if cell not in Q_test_in_true:
                    Q_test_in_true[cell] = x_true.squeeze().numpy()
                Q_test_in_pred_5_all[fold][cell].append(x_pred.squeeze().numpy())
                std_test_in_pred_5_all[fold][cell].append(np.array(std_pred))

with open('Results/Q_test_in_pred_5_all.pkl', 'wb') as f:
    pkl.dump(Q_test_in_pred_5_all, f)

with open('Results/std_test_in_pred_5_all.pkl', 'wb') as f:
    pkl.dump(std_test_in_pred_5_all, f)

with open('Results/Q_test_in_true.pkl', 'wb') as f:
    pkl.dump(Q_test_in_true, f)



Fold 1: Ensemble 1/5
Fold 1: Ensemble 2/5
Fold 1: Ensemble 3/5
Fold 1: Ensemble 4/5
Fold 1: Ensemble 5/5
Fold 2: Ensemble 1/5
Fold 2: Ensemble 2/5
Fold 2: Ensemble 3/5
Fold 2: Ensemble 4/5
Fold 2: Ensemble 5/5
Fold 3: Ensemble 1/5
Fold 3: Ensemble 2/5
Fold 3: Ensemble 3/5
Fold 3: Ensemble 4/5
Fold 3: Ensemble 5/5
Fold 4: Ensemble 1/5
Fold 4: Ensemble 2/5
Fold 4: Ensemble 3/5
Fold 4: Ensemble 4/5
Fold 4: Ensemble 5/5
Fold 5: Ensemble 1/5
Fold 5: Ensemble 2/5
Fold 5: Ensemble 3/5
Fold 5: Ensemble 4/5
Fold 5: Ensemble 5/5
Fold 6: Ensemble 1/5
Fold 6: Ensemble 2/5
Fold 6: Ensemble 3/5
Fold 6: Ensemble 4/5
Fold 6: Ensemble 5/5
Fold 7: Ensemble 1/5
Fold 7: Ensemble 2/5
Fold 7: Ensemble 3/5
Fold 7: Ensemble 4/5
Fold 7: Ensemble 5/5
Fold 8: Ensemble 1/5
Fold 8: Ensemble 2/5
Fold 8: Ensemble 3/5
Fold 8: Ensemble 4/5
Fold 8: Ensemble 5/5
Fold 9: Ensemble 1/5
Fold 9: Ensemble 2/5
Fold 9: Ensemble 3/5
Fold 9: Ensemble 4/5
Fold 9: Ensemble 5/5
Fold 10: Ensemble 1/5
Fold 10: Ensemble 2/5
Fold 10: En

In [ ]:
for fold in range(10):
    Q_test_out_pred_5_all[fold] = {}
    std_test_out_pred_5_all[fold] = {}
    for i in range(5):
        print(f"Fold {fold+1}: Ensemble {i+1}/{num_ensemble}")
        # Initialize the model
        model = Uncertain_LSTM_new(
            num_in=input_size,
            num_augment=num_augment,
            num_hidden=hidden_size,
            seq_len=sequence_length,
            n_layers=num_layers,
            dropout=dropout,
            num_hidden_lstm=hidden_size_lstm,
        )
        # Load the trained model parameters
        model.load_state_dict(model_state_dict[fold][i])
        
        model.eval()
        with torch.no_grad():
            # Evaluate the model on the training set
            for j in range(test_out_X_lstm_eval.shape[0]):
                # Indexing the cell
                cell = test_out_cells[j]
                if cell not in Q_test_out_pred_5_all[fold]:
                    Q_test_out_pred_5_all[fold][cell] = []
                    std_test_out_pred_5_all[fold][cell] = []
                
                # Get the observed capacity
                x = torch.tensor(test_out_X_lstm_eval[j:j+1], dtype=torch.float32)
                x_pred = x.clone()
                # Get predicted remaining capacity
                Q_pred = []
                std_pred = []
                for k in range(len(test_out_remaining_eval[cell])):
                    # Find the features with the corresponding Ah-throughput location
                    c = torch.tensor(test_out_C_lstm_eval[cell][k].reshape(1,-1), dtype=torch.float32)
                    # Predict the remaining capacity ratio and variance
                    seq = x_pred[:,-10:,:]
                    mean_scaled, var_scaled = model(seq, c) # outputs are (ratio between existing and new capacity,  variance)
                    # Squeeze the tensor to remove the batch dimension
                    mean_scaled = mean_scaled.squeeze()
                    all_pred_samples[fold].append(mean_scaled.item())
                    var_scaled = var_scaled.squeeze()
                    # Inverse transform the scaled ratio to the original scale
                    ratio = torch.tensor(capacity_output_scaler.inverse_transform(np.array(mean_scaled.item()).reshape(-1,1)), dtype=torch.float32)
                    var_ratio = var_scaled*(min_max_range**2)
                    # Append the predicted remaining capacity ratio
                    x_pred = torch.cat((x_pred, ratio[:]*(x_pred[:,-1,-1])[None,:,None]), dim=1)
                    # Append the predicted standard deviation
                    std_pred.append(np.sqrt(var_ratio.item())*(x_pred[:,-1,-1])[None,:,None].item())

                # Concatenate the true remaining capacity with the observed capacity
                x_true = torch.concat((x, torch.tensor(test_out_remaining_eval[cell], dtype=torch.float32)[None,:,None]), dim=1)
                if cell not in Q_test_out_true:
                    Q_test_out_true[cell] = x_true.squeeze().numpy()
                Q_test_out_pred_5_all[fold][cell].append(x_pred.squeeze().numpy())
                std_test_out_pred_5_all[fold][cell].append(np.array(std_pred))

with open('Results/Q_test_out_pred_5_all.pkl', 'wb') as f:
    pkl.dump(Q_test_out_pred_5_all, f)

with open('Results/std_test_out_pred_5_all.pkl', 'wb') as f:
    pkl.dump(std_test_out_pred_5_all, f)

with open('Results/Q_test_out_true.pkl', 'wb') as f:
    pkl.dump(Q_test_out_true, f)


Fold 1: Ensemble 1/5
Fold 1: Ensemble 2/5
Fold 1: Ensemble 3/5
Fold 1: Ensemble 4/5
Fold 1: Ensemble 5/5
Fold 2: Ensemble 1/5
Fold 2: Ensemble 2/5
Fold 2: Ensemble 3/5
Fold 2: Ensemble 4/5
Fold 2: Ensemble 5/5
Fold 3: Ensemble 1/5
Fold 3: Ensemble 2/5
Fold 3: Ensemble 3/5
Fold 3: Ensemble 4/5
Fold 3: Ensemble 5/5
Fold 4: Ensemble 1/5
Fold 4: Ensemble 2/5
Fold 4: Ensemble 3/5
Fold 4: Ensemble 4/5
Fold 4: Ensemble 5/5
Fold 5: Ensemble 1/5
Fold 5: Ensemble 2/5
Fold 5: Ensemble 3/5
Fold 5: Ensemble 4/5
Fold 5: Ensemble 5/5
Fold 6: Ensemble 1/5
Fold 6: Ensemble 2/5
Fold 6: Ensemble 3/5
Fold 6: Ensemble 4/5
Fold 6: Ensemble 5/5
Fold 7: Ensemble 1/5
Fold 7: Ensemble 2/5
Fold 7: Ensemble 3/5
Fold 7: Ensemble 4/5
Fold 7: Ensemble 5/5
Fold 8: Ensemble 1/5
Fold 8: Ensemble 2/5
Fold 8: Ensemble 3/5
Fold 8: Ensemble 4/5
Fold 8: Ensemble 5/5
Fold 9: Ensemble 1/5
Fold 9: Ensemble 2/5
Fold 9: Ensemble 3/5
Fold 9: Ensemble 4/5
Fold 9: Ensemble 5/5
Fold 10: Ensemble 1/5
Fold 10: Ensemble 2/5
Fold 10: En

In [ ]:
# Ensemble the predictions
for fold in range(10):
    Q_train_pred_5_ensemble[fold] = {}
    std_train_pred_5_ensemble[fold] = {}
    for cell in training_cells:
        Q_train_pred_5_all[fold][cell] = np.array(Q_train_pred_5_all[fold][cell])
        Q_train_pred_5_ensemble[fold][cell] = np.mean(Q_train_pred_5_all[fold][cell], axis=0)[10:]
        std_train_pred_5_ensemble[fold][cell] = np.sqrt(
            np.mean(np.square(std_train_pred_5_all[fold][cell])
                    +Q_train_pred_5_all[fold][cell][:,10:]**2-Q_train_pred_5_ensemble[fold][cell]**2, axis=0))
        
    Q_test_in_pred_5_ensemble[fold] = {}
    std_test_in_pred_5_ensemble[fold] = {}
    for cell in test_in_cells:
        Q_test_in_pred_5_all[fold][cell] = np.array(Q_test_in_pred_5_all[fold][cell])
        Q_test_in_pred_5_ensemble[fold][cell] = np.mean(Q_test_in_pred_5_all[fold][cell], axis=0)[10:]
        std_test_in_pred_5_ensemble[fold][cell] = np.sqrt(
            np.mean(np.square(std_test_in_pred_5_all[fold][cell])
                    +Q_test_in_pred_5_all[fold][cell][:,10:]**2-Q_test_in_pred_5_ensemble[fold][cell]**2, axis=0))
        
    Q_test_out_pred_5_ensemble[fold] = {}
    std_test_out_pred_5_ensemble[fold] = {}
    for cell in test_out_cells:
        Q_test_out_pred_5_all[fold][cell] = np.array(Q_test_out_pred_5_all[fold][cell])
        Q_test_out_pred_5_ensemble[fold][cell] = np.mean(Q_test_out_pred_5_all[fold][cell], axis=0)[10:]
        std_test_out_pred_5_ensemble[fold][cell] = np.sqrt(
            np.mean(np.square(std_test_out_pred_5_all[fold][cell])
                    +Q_test_out_pred_5_all[fold][cell][:,10:]**2-Q_test_out_pred_5_ensemble[fold][cell]**2, axis=0))
        
with open('Results/Q_train_pred_5_ensemble.pkl', 'wb') as f:
    pkl.dump(Q_train_pred_5_ensemble, f)

with open('Results/std_train_pred_5_ensemble.pkl', 'wb') as f:
    pkl.dump(std_train_pred_5_ensemble, f)

with open('Results/Q_test_in_pred_5_ensemble.pkl', 'wb') as f:
    pkl.dump(Q_test_in_pred_5_ensemble, f)

with open('Results/std_test_in_pred_5_ensemble.pkl', 'wb') as f:
    pkl.dump(std_test_in_pred_5_ensemble, f)

with open('Results/Q_test_out_pred_5_ensemble.pkl', 'wb') as f:
    pkl.dump(Q_test_out_pred_5_ensemble, f)

with open('Results/std_test_out_pred_5_ensemble.pkl', 'wb') as f:
    pkl.dump(std_test_out_pred_5_ensemble, f)